# Error Function of Fabry-Perot Cavity

In [1]:
import numpy as np
import scipy as sci
import scipy.special as sp
import matplotlib as mp
import matplotlib.pyplot as plt 

from scipy.constants import c

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Important functions

In [2]:
# Free spectral range of cavity
def Dnu(L):
    nu = c/(2*L)
    return nu

# Reflaction Coefficient of cavity
def F(om, L, r, conj=False):
    if conj == False:
        i = 1j
    elif conj == True:
        i = -1j
    E = np.exp(i*om/Dnu(L))
    Eref = r*(E - 1)
    Einc = 1 - r**2 * E
    return Eref/Einc

# Power in carrier
def Pc(beta, P0):
    Pc = P0 * (sp.j0(beta))**2
    return Pc

# Power in the sidebands
def Ps(beta, P0):
    Ps = P0 * (sp.j1(beta))**2
    return Ps
    
# Error function
# mod = 1 for fast
# mod = 0 for slow
def err(om, Om, beta, L, r=0.995, om_spacing=1, P0=1, mod=1, approx=False):
    ind = F(om, L, r) * F(om+Om, L, r, conj=True) - F(om, L, r, conj=True) * F(om-Om, L, r)
    norm = np.sqrt(Pc(beta, P0) * Ps(beta, P0))
    if mod==1: # Fast modulation 
        err = -2 * norm * np.imag(ind)
    elif mod==0: # Slow modulation
        if approx == False:
            err = 2 * norm * np.real(ind)
        elif approx == True:
            F2 = np.abs(F(om, L, r))**2
            dF = np.gradient(F2, om_spacing)
            err = P0 * dF * Om * beta
    return err

In [3]:
# Frequency range (Hz) and angular frequency range (rad/s)
freq, freq_spacing = np.linspace(-5e8, 5e8, 400, retstep=True)
w, w_spacing = 2*np.pi*freq, 2*np.pi*freq_spacing

# Length of cavity (m) and reflection coefficint of mirrors
Length, Ref = 0.05, 0.995

# Frequency modulation angular frequency and amplitude
W, Beta = 2*np.pi*Dnu(Length)*0.04, 1

# Power of incoming beam
Power = 1

# Plot it out
Freq = w/Dnu(Length)

norm = (2 * np.sqrt(Pc(Beta, Power) * Ps(Beta, Power)))
Err0 = err(w, W, Beta, Length, Ref, w_spacing, Power, mod=0, approx=False) / norm
Err1 = err(w, W, Beta, Length, Ref, w_spacing, Power, mod=1, approx=False) / norm

fig, ax = plt.subplots(1, 2, figsize=(8, 3))

ax[0].plot(Freq,Err0)
ax[0].set_title('Slow modulation frequency error signal')
ax[0].set_xlabel(r'Frequency ($\omega / \Delta \nu_{fsr}$)')
ax[0].set_ylabel(r'$\epsilon / (2 \sqrt{P_c P_s})$')
ax[0].grid()

ax[1].plot(Freq,Err1)
ax[1].set_title('Fast modulation frequency error signal')
ax[1].set_xlabel(r'Frequency ($\omega / \Delta \nu_{fsr}$)')
ax[1].set_ylabel(r'$\epsilon / (2 \sqrt{P_c P_s})$')
ax[1].grid()

plt.tight_layout()
plt.show()

<Figure size 576x216 with 2 Axes>